In [1]:
import pandas as pd
df = pd.read_pickle("reviews.pkl")
def preprocessing(df):
    df.drop(
        columns=[
            "images",
            "language",
            "url",
            "userId",
            "index",
            "restaurantId",
            "reviewId",
        ],
        inplace=True,
    )
    df["date"] = df["date"].apply(lambda x: x.split()[::2])
    mes = {
        "enero": "01",
        "febrero": "02",
        "marzo": "03",
        "abril": "04",
        "mayo": "05",
        "junio": "06",
        "julio": "07",
        "agosto": "08",
        "septiembre": "09",
        "octubre": "10",
        "noviembre": "11",
        "diciembre": "12",
    }
    df["mes"] = df["date"].apply(lambda x: mes[x[1]])
    df["date"] = df[["date", "mes"]].apply(
        lambda x: str(x[0][0]) + "-" + str(x[1]) + "-" + str(x[0][2]), axis=1
    )
    df.drop(columns=["mes"], inplace=True)
    df["date"] = pd.to_datetime(df["date"])
    return df
df = preprocessing(df)

C:\Users\jilai\AppData\Local\Temp\ipykernel_17784\953578587.py:36: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df["date"])


In [2]:
df_neg = df[df['rating'] < 30]
df_pos = df[df['rating'] > 30]
print('Quantity of negative reviews: ', len(df_neg))
print('Quantity of positive reviews: ', len(df_pos))

Quantity of negative reviews:  89633
Quantity of positive reviews:  458710


In [3]:
df_neg2 = df_neg.sample(n=10_000)
df_pos2 = df_pos.sample(n=10_000)

In [4]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from sklearn.feature_extraction.text import CountVectorizer

representation_model = KeyBERTInspired()
#vectorizer_model = CountVectorizer(min_df=5, stop_words = 'english')

topic_model_neg = BERTopic( language="multilingual", #nr_topics = 'auto', 
 #vectorizer_model = vectorizer_model,
 representation_model = representation_model)

topic_model_pos = BERTopic( language="multilingual", #nr_topics = 'auto', 
 #vectorizer_model = vectorizer_model,
 representation_model = representation_model)

In [5]:
print('Training topic model for negative reviews...')
topics_neg, ini_probs_neg = topic_model_neg.fit_transform(list(df_neg2["text"].values))

print('Training topic model for positive reviews...')
topics_pos, ini_probs_pos = topic_model_pos.fit_transform(list(df_pos2["text"].values))

df_neg2['topic'] = topics_neg
df_neg2['topic_prob'] = ini_probs_neg
df_pos2['topic'] = topics_pos
df_pos2['topic_prob'] = ini_probs_pos

topics_info_neg = topic_model_neg.get_topic_info()
topics_info_pos = topic_model_pos.get_topic_info()

Training topic model for negative reviews...
Training topic model for positive reviews...


In [6]:
topic_model_neg.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,9773,0_restaurante_camareros_fuimos_camarero,"[restaurante, camareros, fuimos, camarero, com...",[Unos amigos que estaban celebrando el cumplea...
1,1,227,1_pizza_pizzas_pizzería_restaurante,"[pizza, pizzas, pizzería, restaurante, italian...","[No me gusta escribir críticas negativas, pero..."


In [7]:
topic_model_pos.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4466,-1_restaurante_cenar_camareros_postre,"[restaurante, cenar, camareros, postre, comida...",[Fui con unos amigos y la verdad me gustó. El ...
1,0,1305,0_madrid_restaurantes_restaurante_lugar,"[madrid, restaurantes, restaurante, lugar, don...",[No sé si se trata del mejor mexicano de Madri...
2,1,582,1_agradable_bonito_lugar_disfrutar,"[agradable, bonito, lugar, disfrutar, excelent...",[Un sitio para ir de tapas o comer un buen men...
3,2,457,2_agradecemos_alegramos_gracias_saludos,"[agradecemos, alegramos, gracias, saludos, res...",[El servicio es excelente. La presentación tam...
4,3,245,3_hamburguesas_hamburguesa_burger_steakburger,"[hamburguesas, hamburguesa, burger, steakburge...","[Tienen una buena carta de hamburguesas, y su ..."
...,...,...,...,...,...
60,59,11,59_deliciosa_afortunadamente_excelente_buenrol...,"[deliciosa, afortunadamente, excelente, buenro...",[No me voy a dejar influir por las demás opini...
61,60,11,60_cenando_disfrutamos_cenar_memorable,"[cenando, disfrutamos, cenar, memorable, gustó...",[Fui con 3 amigas a cenar y estuvo muy bien. Y...
62,61,11,61_camarero_agradable_cenamos_ocupamos,"[camarero, agradable, cenamos, ocupamos, atend...",[He estado varias veces a lo largo de los años...
63,62,11,62_restaurante_comida_buena_calidad,"[restaurante, comida, buena, calidad, confecci...",[Este restaurante destacada por la originalida...


In [8]:
topic_model_neg.visualize_barchart()

In [9]:
topic_model_pos.visualize_barchart()

In [27]:
df_neg3 = pd.DataFrame(topic_model_neg.get_topic_info())

In [28]:
df_pos3 = pd.DataFrame(topic_model_pos.get_topic_info())

In [31]:
df_neg3.to_pickle('df_neg.pkl')
df_pos3.to_pickle('df_pos.pkl')